In [3]:
!pip install ultralytics opencv-python --user

  Using cached ultralytics-8.3.160-py3-none-any.whl.metadata (37 kB)
  Using cached matplotlib-3.10.3-cp312-cp312-win_amd64.whl.metadata (11 kB)
  Using cached pillow-11.2.1-cp312-cp312-win_amd64.whl.metadata (9.1 kB)
  Using cached scipy-1.16.0-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached torch-2.7.1-cp312-cp312-win_amd64.whl.metadata (28 kB)
  Using cached torchvision-0.22.1-cp312-cp312-win_amd64.whl.metadata (6.1 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl.metadata (794 bytes)
  Using cached pandas-2.3.0-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached ultralytics_thop-2.0.14-py3-none-any.whl.metadata (9.4 kB)
  Using cached contourpy-1.3.2-cp312-cp312-win_amd64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.58.4-cp312-cp312-win_amd64.whl.metadata (108 kB)
  Using cached kiwisolver-1.4.8-cp312-cp312-win_amd64.whl.metadata (6.3 

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [4]:
import cv2
from ultralytics import YOLO

# YOLOv11 Pose modelini yükle
model = YOLO('yolo11n-pose.pt')  # Yol değişebilir, doğru model dosyasını kullan

# Webcam'i başlat
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Model ile tahmin yap
    results = model.predict(source=frame, conf=0.5, verbose=False)

    # Tahminleri çiz
    annotated_frame = results[0].plot()

    # Sonucu göster
    cv2.imshow("Pose Estimation", annotated_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [16]:
import cv2
from ultralytics import YOLO
import numpy as np
import math

def spine_deviation_angle(shoulder_ctr, hip_ctr):
    """
    Omuz merkezinden kalça merkezine giden vektörün
    dikey eksene (y-ekseni) göre sapma açısını verir.
    """
    vec = hip_ctr - shoulder_ctr
    dx, dy = vec[0], vec[1]
    # OpenCV y-ekseni aşağı olduğu için dy mutlak; dx mutlak
    angle = math.degrees(math.atan2(abs(dx), abs(dy)))
    return angle

# Modeli yükle
model = YOLO('yolo11n-pose.pt')  # veya kendi model dosyan

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model.predict(source=frame, conf=0.5, verbose=False)
    annotated = results[0].plot()
    keypoints = results[0].keypoints

    if keypoints is not None and len(keypoints.xy) > 0:
        kp = keypoints.xy[0].cpu().numpy()
        # 5=sol omuz, 6=sağ omuz, 11=sol kalça, 12=sağ kalça
        left_sh, right_sh = kp[5], kp[6]
        left_hp, right_hp = kp[11], kp[12]
        sh_ctr = (left_sh + right_sh) / 2
        hp_ctr = (left_hp + right_hp) / 2

        angle = spine_deviation_angle(sh_ctr, hp_ctr)
        threshold = 15  # dilersen 10 veya 20 yap ve teste bak

        if angle > threshold:
            text = f"Wrong Posture! {int(angle)}°"
            color = (0, 0, 255)
        else:
            text = f"Good Posture. {int(angle)}°"
            color = (0, 255, 0)

        cv2.putText(annotated, text, (50, 100),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.5, color, 3)

    annotated = cv2.resize(annotated, (1920, 1080))
    cv2.imshow("Pose Estimation", annotated)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [1]:
import cv2
import time
import math
import numpy as np
import pandas as pd
from ultralytics import YOLO

# ─── Açı Hesaplama Fonksiyonları ─────────────────────────────────────

def spine_angle(sh_ctr, hp_ctr):
    """Omuz ortası–kalça ortası doğrultusunun dikey eksene sapma açısı."""
    dx, dy = hp_ctr - sh_ctr
    return math.degrees(math.atan2(abs(dx), abs(dy)))

def neck_angle(sh_ctr, head_pt):
    """Omuz ortası–baş noktası doğrultusunun dikey eksene sapma açısı."""
    dx, dy = head_pt - sh_ctr
    return math.degrees(math.atan2(abs(dx), abs(dy)))

def shoulder_tilt(l_sh, r_sh):
    """Sol ve sağ omuz noktaları arasındaki hattın yatay eksene sapma açısı."""
    dx = r_sh[0] - l_sh[0]
    dy = r_sh[1] - l_sh[1]
    return math.degrees(math.atan2(abs(dy), abs(dx)))

# ─── Kalibrasyon Fonksiyonu ──────────────────────────────────────────

def calibrate(model, cap, secs=3):
    """
    Kullanıcı ideal pozda iken 3 saniye boyunca
    spine/neck/tilt açılarını toplayıp medyan baseline değerleri döner.
    """
    print(f"[Kalibrasyon] Lütfen {secs} saniye boyunca ideal pozda kalın...")
    frames = int(secs * 30)  # ~30 FPS
    vals = {"spine": [], "neck": [], "tilt": []}
    count = 0

    while count < frames:
        ret, frame = cap.read()
        if not ret:
            continue
        res = model.predict(source=frame, conf=0.5, verbose=False)[0]
        kp = res.keypoints
        if kp is None or len(kp.xy) == 0:
            continue

        pts = kp.xy[0].cpu().numpy()
        l_sh, r_sh = pts[5], pts[6]
        l_hp, r_hp = pts[11], pts[12]
        nose = pts[0]

        sh_ctr = (l_sh + r_sh) / 2
        hp_ctr = (l_hp + r_hp) / 2

        vals["spine"].append(spine_angle(sh_ctr, hp_ctr))
        vals["neck"].append(neck_angle(sh_ctr, nose))
        vals["tilt"].append(shoulder_tilt(l_sh, r_sh))

        count += 1

    baseline = {k: float(np.median(v)) for k, v in vals.items()}
    print("[Kalibrasyon tamam] Baseline değerleri:", baseline)
    return baseline

# ─── Ana Program ────────────────────────────────────────────────────

# Modeli yükle
model = YOLO('yolo11n-pose.pt')

# Kamera ayarları
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)

# 1) Kalibrasyon
baseline = calibrate(model, cap, secs=3)

# 2) Dinamik tolerans değerleri (delta)
delta = {
    "spine": max(baseline["spine"] * 0.3, 10),
    "neck":  max(baseline["neck"]  * 0.3,  8),
    "tilt":  max(baseline["tilt"]  * 0.3,  5)
}
print("Delta değerleri:", delta)

# 3) Veri toplama listeleri
timestamps = []
labels     = []

# Tam ekran pencere
cv2.namedWindow("Posture Checker", cv2.WINDOW_NORMAL)
cv2.setWindowProperty("Posture Checker", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    t0 = time.time()
    res = model.predict(source=frame, conf=0.5, verbose=False)[0]
    annotated = res.plot()
    kp = res.keypoints

    label = "No Person"
    if kp is not None and len(kp.xy) > 0:
        pts = kp.xy[0].cpu().numpy()
        l_sh, r_sh = pts[5], pts[6]
        l_hp, r_hp = pts[11], pts[12]
        nose = pts[0]

        sh_ctr = (l_sh + r_sh) / 2
        hp_ctr = (l_hp + r_hp) / 2

        # Açı hesaplama
        s = spine_angle(sh_ctr, hp_ctr)
        n = neck_angle(sh_ctr, nose)
        t = shoulder_tilt(l_sh, r_sh)

        # Normalize edilmiş sapma skorları
        ds = abs(s - baseline["spine"]) / delta["spine"]
        dn = abs(n - baseline["neck"])  / delta["neck"]
        dt = abs(t - baseline["tilt"])  / delta["tilt"]

        # Bileşik Posture skoru (0–1 arası)
        score = 1 - np.clip((ds + dn + dt) / 3, 0, 1)

        # Etiket belirleme
        if score > 0.85:
            label = "Good"
        elif score > 0.5:
            label = "Fair"
        else:
            label = "Bad"

        # Ekrana yazdırma
        pct = int(score * 100)
        color = (0,255,0) if label=="Good" else ((0,255,255) if label=="Fair" else (0,0,255))
        cv2.putText(annotated, f"{label} ({pct}%)", (50,80),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.5, color, 3)
        cv2.putText(annotated,
            f"S:{s:.1f}° N:{n:.1f}° T:{t:.1f}°",
            (50,150), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255,255,255), 2
        )

    # Veriyi kaydet
    timestamps.append(t0)
    labels.append(label)

    cv2.imshow("Posture Checker", annotated)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# ─── Sonuçların Özetlenmesi ──────────────────────────────────────────

# DataFrame oluştur
df = pd.DataFrame({
    "timestamp": pd.to_datetime(timestamps, unit='s'),
    "label": labels
})

# Her frame arası süre farkını hesapla
df['delta_s'] = df['timestamp'].diff().dt.total_seconds().fillna(0)

# Label bazında toplam süre
summary = df.groupby('label')['delta_s'].sum().reset_index()
summary.columns = ['Posture', 'Duration_s']
summary['Duration_min'] = (summary['Duration_s'] / 60).round(2)

print("\n=== Posture Süre Özeti ===")
print(summary.to_string(index=False))


[Kalibrasyon] Lütfen 3 saniye boyunca ideal pozda kalın...
[Kalibrasyon tamam] Baseline değerleri: {'spine': 6.002932015691678, 'neck': 34.14485492575423, 'tilt': 2.9072766655301274}
Delta değerleri: {'spine': 10, 'neck': 10.24345647772627, 'tilt': 5}

=== Posture Süre Özeti ===
Posture  Duration_s  Duration_min
    Bad   82.635919          1.38
   Fair   26.376210          0.44
   Good    7.514798          0.13


In [25]:
summary

,Posture,Duration_s,Duration_min
0,Bad,13.782007,0.23
1,Fair,24.235121,0.40
2,Good,23.001549,0.38
